# 尝试使用各种方案进行  两大处理， 样本不均衡问题和   半监督学习方式的使用

In [1]:
#################     引包   #####################
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import matplotlib
import matplotlib.pyplot as plt
import lightgbm as lgb
import operator
import time
import xgboost as xgb

%matplotlib inline

# 读取文件
train_xy = pd.read_csv("../data/train_xy.csv",header=0,sep=",")
train_x = pd.read_csv("../data/train_x.csv",header=0,sep=",")
test_all = pd.read_csv("../data/test_all.csv",header=0,sep=",")
print(train_xy.shape)
print(train_x.shape)
print(test_all.shape)
train = train_xy.copy()
test = test_all.copy()
test['y'] = -1
print('训练集的特征列train：',train.columns)
print('测试集的特征列test：',test.columns)
# 合并一下train 和 test
data = pd.concat([train,test],axis = 0) # train_xy，test_all索引上连接
print(train.shape)
print(test.shape)
print(data.shape)
print('训练集的特征列：',data.columns)

(15000, 160)
(10000, 159)
(10000, 159)
训练集的特征列train： Index(['cust_id', 'cust_group', 'y', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6',
       'x_7',
       ...
       'x_148', 'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155',
       'x_156', 'x_157'],
      dtype='object', length=160)
训练集的特征列test： Index(['cust_id', 'cust_group', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6',
       'x_7', 'x_8',
       ...
       'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155', 'x_156',
       'x_157', 'y'],
      dtype='object', length=160)
(15000, 160)
(10000, 160)
(25000, 160)
训练集的特征列： Index(['cust_group', 'cust_id', 'x_1', 'x_10', 'x_100', 'x_101', 'x_102',
       'x_103', 'x_104', 'x_105',
       ...
       'x_91', 'x_92', 'x_93', 'x_94', 'x_95', 'x_96', 'x_97', 'x_98', 'x_99',
       'y'],
      dtype='object', length=160)


d:\python35\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [2]:
train_id = train.pop('cust_id')
y = train['y'].values
print(train.columns)
X = train[features].values
print('X features :',features)
print("X shape:",X.shape)
print("y shape:",y.shape)

test_id = test.pop('cust_id')
test_data = test[features].values
print("test shape",test_data.shape)
print(train.columns)
#train.to_csv('./feature_data/经过特征工程后的整个表.csv')
# X.to_csv('./feature_data/train_feature.csv')
# y.to_csv('./feature_data/train_label.csv')
print('训练集的特征列：',train.columns)

#使用最基本特征跑一次得分
#开始xgb的cv训练
# 采取分层采样
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
print("start：********************************")
start = time.time()


N = 5
skf = StratifiedKFold(n_splits=N,shuffle=True,random_state=2018)

auc_cv = []
pred_cv = []
for k,(train_in,test_in) in enumerate(skf.split(X,y)):
    X_train,X_test,y_train,y_test = X[train_in],X[test_in],\
                                    y[train_in],y[test_in]    
    # 模型参数
    params = {	'booster':'gbtree',
	'objective': 'binary:logistic',
	'early_stopping_rounds':100,
#     'gamma':0,#0.2 is ok
#     'max_depth':8,
# # 	'lambda':550,
    'subsample':0.7,
    'colsample_bytree':0.3,
    'min_child_weight':2.5, 
    'eta': 0.007,
	'seed':2018,
	'nthread':7}
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test)
    model = xgb.train(params, dtrain, num_boost_round=800)
    predict = model.predict(dvali)
    tmp_auc = roc_auc_score(y_test,predict)
    auc_cv.append(tmp_auc)
    print("valid auc:",tmp_auc)
    # test
    pred = model.predict(xgb.DMatrix(test_data))
    pred_cv.append(pred) 
    
    
#     xgb.plot_importance(model,max_num_features=20)
#     plt.title('xgb Feature Importance')
#     plt.xlabel('relative importance')
#     plt.show()

# K交叉验证的平均分数 
print('the cv information:')
print(auc_cv)
print('cv mean score',np.mean(auc_cv))

end = time.time()
print("......................run with time: ",(end - start) / 60.0 )
print("over:*********************************")

Index(['cust_group', 'y', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7',
       'x_8',
       ...
       'x_148', 'x_149', 'x_150', 'x_151', 'x_152', 'x_153', 'x_154', 'x_155',
       'x_156', 'x_157'],
      dtype='object', length=159)


NameError: name 'features' is not defined

# 第一块  样本不均衡问题

# 第二块  半监督学习，样本利用问题

# 方案一


    使用最好的模型对样本进行预测，迭代找到合适的阈值w，让其满足在一个组内的正负样本比例满足组上的关系：
       *对于组1：满足正负比10:1
       *对于组2:满足正负比35:1
       *对于组2:满足正负比45:1

# 方案二
      使用传统过采样、权值调整发进行处理